#### Install Anthropic's SDK

In [ ]:
%pip install anthropic


#### Use ipython store to manage your Anthropic API_KEY, MODEL_NAME
- You can get your api key from your console

In [452]:
API_KEY="your_api_key"
MODEL_NAME="claude-3-7-sonnet-20250219"
%store API_KEY
%store MODEL_NAME

Stored 'API_KEY' (str)
Stored 'MODEL_NAME' (str)


##### Additional Python Store functionalities
- Should you need to retrieve these keys in another python notebook you can uncomment the following code and run it in your notebook:

In [ ]:
#%store -r API_KEY
#%store -r MODEL_NAME

#### Importing the data and other related dependencies

In [ ]:
import sqlite3
import json
db_path = r"C:\Users\badhw\Downloads\nba.sqlite" # replace with your db_path
json_path=r"C:\Users\badhw\Downloads\ground_truth_data.json" # replace with your json_path

In [454]:
#loading ground truth data from the json file
with open(json_path, "r", encoding="utf-8") as f:
    ground_truth_data = json.load(f)

In [440]:
# check if data is loaded by printing top 5 rows.
ground_truth_data[:5]

[{'natural_language': 'How many teams are in the NBA?',
  'sql': 'SELECT COUNT(*) as team_count FROM team LIMIT 1',
  'type': 'counting'},
 {'natural_language': 'What are the 5 oldest teams in the NBA?',
  'sql': 'SELECT full_name FROM team ORDER BY year_founded ASC LIMIT 5',
  'type': 'ranking'},
 {'natural_language': 'List all teams from California',
  'sql': "SELECT full_name FROM team WHERE state = 'California'",
  'type': 'filtering'},
 {'natural_language': "What's the total number of games played?",
  'sql': 'SELECT COUNT(DISTINCT game_id) as total_games FROM game LIMIT 1',
  'type': 'counting'},
 {'natural_language': "What's the highest scoring game?",
  'sql': 'SELECT g.pts_home + g.pts_away as total_points FROM game g ORDER BY total_points DESC LIMIT 1',
  'type': 'ranking'}]

In [455]:
# checking number of ground truth data entries
len(ground_truth_data)

98

In [456]:
# Extract unique types of queries and their value counts
query_type_counts = {}
for i in ground_truth_data:
    query_type = i.get("type")
    query_type_counts[query_type] = query_type_counts.get(query_type, 0) + 1

query_type_counts

{'counting': 19,
 'ranking': 30,
 'filtering': 13,
 'aggregation': 25,
 'detail': 7,
 'comparison': 3,
 'history': 1}

#### Setting up the API call to Claude
- We will use this function to interact with Claude
- For NL-> SQL tasks, Claude 3.7 Sonnet is one Anthropic's best performing models.
- However, should you want to change the model, you can always refer to our model codes and change the MODEL_NAME variable at the top of this notebook



In [457]:
import anthropic
client = anthropic.Anthropic(api_key=API_KEY)
def api_call(prompt:str):
    message=client.messages.create(
        model=MODEL_NAME, # using Claud 3.7
        max_tokens=2000, # good range for NL->SQL tasks
        temperature=0.0, # more deterministic
        messages=[{"role":"user","content":prompt}] # standard claude api message template always starts with user, can be continued with a user-assistant-user-... format
    )
    return message.content[0].text

#### Function to run the ground truth queries on the DB
- Arguments: The list of dictionaries which has the ground truth question-SQL query pairs
- Returns: None
- What it does: The function runs the ground truth SQL on the DB and then adds the ground truth result of the SQL into the ground truth json.

In [458]:
def run_ground_truth_query(input_data,db_path):

    for sample in input_data:
        sql = sample["sql"]
        try:
            conn= sqlite3.connect(db_path)
            cursor = conn.cursor()
            cursor.execute(sql)
            results = cursor.fetchall()
            conn.close()
            
            # below code flattens the SQL result into a list depending upon the query output
            if len(results) > 0 and len(results[0]) == 1:
                flat = [row[0] for row in results]
            else:
                flat = results
            sample["ground_truth_result"]=flat
        except sqlite3.Error as e:
            sample["ground_truth_result"]=None
    
    return None

#### Running the ground truth queries on the DB
- This can be run just once, and our ground truth SQL output will be populated in the ground turth jason along with the ground truth questions and SQL
- This should take ~7 seconds to run.

In [459]:
run_ground_truth_query(ground_truth_data,db_path)

##### Checking the ground truth data for ground truth SQL outputs
- You can see the ground truth results have been added after running the cell below, (top 5 only)

In [460]:
ground_truth_data[:5]

[{'natural_language': 'How many teams are in the NBA?',
  'sql': 'SELECT COUNT(*) as team_count FROM team LIMIT 1',
  'type': 'counting',
  'ground_truth_result': [30]},
 {'natural_language': 'What are the 5 oldest teams in the NBA?',
  'sql': 'SELECT full_name FROM team ORDER BY year_founded ASC LIMIT 5',
  'type': 'ranking',
  'ground_truth_result': ['Boston Celtics',
   'Golden State Warriors',
   'New York Knicks',
   'Los Angeles Lakers',
   'Sacramento Kings']},
 {'natural_language': 'List all teams from California',
  'sql': "SELECT full_name FROM team WHERE state = 'California'",
  'type': 'filtering',
  'ground_truth_result': ['Golden State Warriors',
   'Los Angeles Clippers',
   'Los Angeles Lakers',
   'Sacramento Kings']},
 {'natural_language': "What's the total number of games played?",
  'sql': 'SELECT COUNT(DISTINCT game_id) as total_games FROM game LIMIT 1',
  'type': 'counting',
  'ground_truth_result': [65642]},
 {'natural_language': "What's the highest scoring game?

#### Function to generate SQL from Claude
- Arguments: list containing ground truth question-SQL pairs and query results, api_call function to claude
- Returns: list of sql queries in the same order as questions in the ground truth.
- Based on Claude's outut, there is also a helper function to extract sql from the output if needed.

In [461]:
import re
def extract_raw_sql(response): # helper function to get raw SQL from claudes output
    match = re.search(r"```sql\s+(.*?)```", response, re.DOTALL | re.IGNORECASE) 
    if match:
        return match.group(1).strip()
    else:
        return None

def generate_sql_from_claude(input_data,prompt_template):

    claude_generated_sql_list=[]
    
    for i in input_data:
        question=i['natural_language'] # extracting the natural language question from the ground truth data
        query_type=i['type'] # extract query type
        prompt_vars = {"question": question,"query_type": query_type} # this is used so that it works for both the original prompt and prompt engineering
        prompt=prompt_template.format(**prompt_vars) # injecting our question, query_type(depending on the prompt) into the prompt template
        claude_sql=api_call(prompt) #generating sql using claude
        claude_generated_sql_list.append(claude_sql)
    
    return claude_generated_sql_list

#### Run the below cell to get a list of SQL queries generated by Claude
- First run the cell with the prompt, which is the original prompt in your Email to me
- Then run the cells below to generate SQL using Claude, that will print Claude's response to the first 5 rows.

In [462]:
original_prompt="""
Convert this question to SQL:
{question}
"""

**Important Note**
- If you'd like to generate the SQL using claude on a 20% random sample of ground truth data, then uncomment the cell below and then run the generate_sql_from_claude function. This will decrease the latency and in production environments - possible rate limiting. **In this case, we make sure to use the ground_truth_sample dataset everywhere else.**
- If you'd like to generate SQL on the entire dataset then run the generate_sql_from_claude function directly. The code provided here does that and will take ~5-6 minutes.

In [ ]:
# import random
# random.seed(42)
# ground_truth_sample = random.sample(ground_truth_data,20)

In [463]:

claude_response=generate_sql_from_claude(ground_truth_data,original_prompt)


- Printing the 5 questions and Claude generated SQL for it.

In [464]:
for i,j in zip(ground_truth_data[:5],claude_response[:5]):
    print("#"*60,f"\n question {i['natural_language']}:\n",j,"\n")

############################################################ 
 question How many teams are in the NBA?:
 ```sql
SELECT COUNT(*) AS total_teams
FROM teams
WHERE league = 'NBA';
```

This SQL query counts the number of teams in the NBA by:
1. Selecting the COUNT of all rows
2. From the teams table
3. Where the league is 'NBA'
4. Returning the result as "total_teams"

Note: This assumes there is a "teams" table with a "league" column that contains the value 'NBA' for NBA teams. If your database has a different structure, the query would need to be adjusted accordingly. 

############################################################ 
 question What are the 5 oldest teams in the NBA?:
 ```sql
SELECT team_name, year_founded
FROM teams
WHERE league = 'NBA'
ORDER BY year_founded ASC
LIMIT 5;
```

This SQL query:
1. Selects the team name and founding year from the teams table
2. Filters for only NBA teams
3. Orders the results by founding year in ascending order (oldest first)
4. Limits the resu

#### We can see that the SQL comes with other results/assumptions Claude generates
- We can run the below cells to just get the raw SQL

In [510]:
def responses_to_sql_list(responses):
    return [extract_raw_sql(r) for r in responses]

claude_sql_raw=responses_to_sql_list(claude_response)
for i,j in zip(ground_truth_data[:5],claude_sql_raw[:5]):
    print("#"*60,f"\n question: {i['natural_language']}\n SQL:\n",j,"\n")

############################################################ 
 question: How many teams are in the NBA?
 SQL:
 SELECT COUNT(*) AS total_teams
FROM teams
WHERE league = 'NBA'; 

############################################################ 
 question: What are the 5 oldest teams in the NBA?
 SQL:
 SELECT team_name, year_founded
FROM teams
WHERE league = 'NBA'
ORDER BY year_founded ASC
LIMIT 5; 

############################################################ 
 question: List all teams from California
 SQL:
 SELECT *
FROM teams
WHERE state = 'California'; 

############################################################ 
 question: What's the total number of games played?
 SQL:
 SELECT COUNT(*) AS total_games_played
FROM games; 

############################################################ 
 question: What's the highest scoring game?
 SQL:
 SELECT TOP 1 *
FROM games
ORDER BY score DESC 



#### Function to run Claude generated SQL on the DB (this is quite similar to the earlier function for ground truth)
- The function below: run_claude_query is called in the final evaluation function (evaluate_sql_accuracy) where the SQL query generated by Claude for every question is run on the database and compared with the ground truth output.
- If the running the SQL on the DB returns an Error, the result is recorded as None

In [511]:
def run_claude_query(sql,db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql)
        results = cursor.fetchall()
        conn.close()
        # flatten query results    
        if len(results) > 0 and len(results[0]) == 1:
            flat = [row[0] for row in results]
        else:
            flat = results
        return flat
    except sqlite3.Error as e:
        return None

#### Function to evaluate the results & accuracy of the Claude generated SQL with the ground truth
- Arguments: Ground truth data, Claude generated sql, db_path
- Returns: Overall accuracy, Dictionary with individual results for each question
#### Notes about this evaluation
- Quantitative evaluation (Semantic query equivalence): We compare the results of the running Claude's queries on the DB with the ground truth SQL results to evaluate SQL semantic similarity.
- Two different queries could product the same result, which is why we're comparing the results
- **Accuracy** =  $\frac{\text{number of events where Claude's SQL output equals ground-truth result}}{\text{total number of questions asked}}$

In [502]:
def evaluate_sql_accuracy(input_data,claude_generated_sqls,db_path):

    detailed_results= [] # list of di tionaries storing details mentioned below in the function
    correct = 0

    for index, data in enumerate(input_data):
        gt_result = data["ground_truth_result"] 
        claude_sql = claude_generated_sqls[index]
        claude_sql_result = run_claude_query(claude_sql,db_path) # run claude generated sql

        is_match = claude_sql_result is not None and claude_sql_result == gt_result # checks results for ground truth and query matching, and for any errors.
        if is_match:
            correct +=1
        
        detailed_results.append(
            {
                "question": data["natural_language"],
                "ground_truth_sql": data["sql"],
                "gorund_truth_query_type": data["type"],
                "claude_sql": claude_sql,
                "sql_semantic_match": is_match,
                "ground_truth": gt_result,
                "claude_sql_result": claude_sql_result,
            }
        )

    accuracy = correct/len(input_data) if data else 0.0
    return {"accuracy": accuracy, "detailed_results": detailed_results}

#### Run below cells to evaluate the original prompt

In [515]:
results=evaluate_sql_accuracy(ground_truth_data,claude_sql_raw,db_path)

#### Top 2 results of the evaluation, this shows the question, ground truth sql and semantic match

In [470]:
for i in results['detailed_results'][:2]:
    print(json.dumps(i, indent=2))
print("Overall accuracy:", round(results['accuracy']*100,2), "%")

{
  "question": "How many teams are in the NBA?",
  "ground_truth_sql": "SELECT COUNT(*) as team_count FROM team LIMIT 1",
  "gorund_truth_query_type": "counting",
  "claude_sql": "SELECT COUNT(*) AS total_teams\nFROM teams\nWHERE league = 'NBA';",
  "sql_semantic_match": false,
  "ground_truth": [
    30
  ],
  "claude_sql_result": null
}
{
  "question": "What are the 5 oldest teams in the NBA?",
  "ground_truth_sql": "SELECT full_name FROM team ORDER BY year_founded ASC LIMIT 5",
  "gorund_truth_query_type": "ranking",
  "claude_sql": "SELECT team_name, year_founded\nFROM teams\nWHERE league = 'NBA'\nORDER BY year_founded ASC\nLIMIT 5;",
  "sql_semantic_match": false,
  "ground_truth": [
    "Boston Celtics",
    "Golden State Warriors",
    "New York Knicks",
    "Los Angeles Lakers",
    "Sacramento Kings"
  ],
  "claude_sql_result": null
}
Overall accuracy: 0.0 %


#### Evaluating the original prompt setup:

- The current prompt is **zero-shot**, which means that Claude recieved a request without any context or examples to refer at all. Before extracting the raw SQL from Claude's responses, we could see that it had made various assumptions about the DB schema, column names and mentioned those assumptions in its response. Those are placeholder tables and will usually not be the same as the actual schema.
- While we did run a result equivalence function here. It had a pre-determined outcome of very little to 0 accuracy, since Claude just generated queries which were based on assumptions
- Qualitatively, we can see that the prompt requests are going through to Claude and it is generating queries which it thinks are the best fit to the question and clearly mentions in some of the responses that you might need to change table/column names appropriately.
- Claude does well for zero shot prompts which might request factual/known information that it is already trained on. However, for mor =e specific requests it needs additional context to do well.
- We can use prompt engineering to produce better accuracy for NL-> SQL for your analytics team

# Improving accuracy by prompt engineering
- We will start with try various techniques to improve the accuracy of the queries we get.
- Since we will be testing out various prompts, we will only use a **~10% random sample** of the ground_truth data to decrease latency.

In [471]:
import random
random.seed(42)
ground_truth_sample = random.sample(ground_truth_data,10)

#### Function to run the entire setup we have done so far, with new prompts, in a single click
- For operational ease, you can simply run the prompt_engineering_result function in the cell below, which takes the argument of the engineered prompt and then prints the results of the evaluation.

In [495]:
def prompt_engineering_result(prompt,data):
    
    claude_sql=generate_sql_from_claude(data,prompt) # Claude generates SQL for each question
    claude_sql_raw=responses_to_sql_list(claude_sql)
    results=evaluate_sql_accuracy(data,claude_sql_raw,db_path) # Claude generated SQL is run on the DB and the final accuracy is returned
    for i in results['detailed_results'][:2]:
        print(json.dumps(i, indent=2))
        print("Overall accuracy:", round(results['accuracy']*100,2), "%")
    return results,claude_sql


#### Recommendations for improving accuracy:

**General recommendations**
- Claude is trained to recognize XML tags, hence a good recommendation is to wrap specific instructions within prompts in XML tags.<br><br>
- Prompts can be more specific by chaining various sub prompts which provide additional context to claude. We have demonstrated this below<br><br>
- Claude can also be asked to think before it responds, this refers to **precognition** and can be a good approach to course correct Claude's responses.<br><br>
- Giving Claude an out: Claude can also be asked to not hallucinate or refer to external resources, which is very important in NL->SQL tasks, this can be done by giving system level instructions.<br><br>
- Few shot prompting: Claude will be more accurate if provided with some examples.<br><br>
- Make sure the prompts are not too verbose, we can find creative ways like defining helper functions to make Claude think a certain way.

**Specific recommendations**

Each of these points will be sub prompts chained together, which will then be passed to Claude.
- system_level_prompt: 
    - Incase Claude lacks enough context to generate SQL, we can give it a system level prompt asking it to not halllucinate. <br><br>
    - We see that in zero shot prompting Claude generates queries which have WHERE LEAGUE='NBA'. We can add a system instruction saying that the DB is only for NBA by default. <br><br>
    - We also define a helper function which Claude can call, to get details about the schema should it not have enough context from the few shot schema

- query_type:
    - We can add <query_type>{QUERY_TYPE}</query_type> variable. This will give claude context about the query type. <br><br>
    - Earlier, we saw that for ranking queries say Top 5 oldest teams, Claude SQL output had Team Name, Year Founded in its output. To avoid this, we can give descriptions for each query type. Example: For <query_type>ranking</query_type> please make a query that outputs names of the entities to be ranked, and nothing else. So on for other query_type that are not obvious, like aggregation. <br><br>

- db_schema_context:
    - Tables to use:
        - Earlier, we saw that Claude hallucinated table names, with our system prompt, it will not. And so it is important to provide a most commonly used table names and their descriptions. These will also be wrapped in XML tags.<br><br>
    - Columns to use: 
        - Similarly, we can define Primary, Foreign keys and their descriptions, this will be important to get accurate column names in generated SQL.

#### You can run below cells to get columns from tables and get insights on the actual DB schema (Optional)

In [518]:
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = run_claude_query(tables_query,db_path)
print("Tables in database:", tables)


Tables in database: ['game', 'game_summary', 'other_stats', 'officials', 'inactive_players', 'game_info', 'line_score', 'player', 'team', 'common_player_info', 'team_details', 'team_history', 'draft_combine_stats', 'draft_history', 'team_info_common']


In [519]:
def get_columns_for_table(table_name, db_path):
    query = f"PRAGMA table_info({table_name});"
    results = run_claude_query(query, db_path)
    return [(col[1], col[2]) for col in results]

In [520]:
for table in tables:
    columns = get_columns_for_table(table, db_path)
    print(f"\nTable: {table}")
    for col_name, col_type in columns:
        print(f"   - {col_name}: {col_type}")


Table: game
   - season_id: TEXT
   - team_id_home: TEXT
   - team_abbreviation_home: TEXT
   - team_name_home: TEXT
   - game_id: TEXT
   - game_date: TIMESTAMP
   - matchup_home: TEXT
   - wl_home: TEXT
   - min: INTEGER
   - fgm_home: REAL
   - fga_home: REAL
   - fg_pct_home: REAL
   - fg3m_home: REAL
   - fg3a_home: REAL
   - fg3_pct_home: REAL
   - ftm_home: REAL
   - fta_home: REAL
   - ft_pct_home: REAL
   - oreb_home: REAL
   - dreb_home: REAL
   - reb_home: REAL
   - ast_home: REAL
   - stl_home: REAL
   - blk_home: REAL
   - tov_home: REAL
   - pf_home: REAL
   - pts_home: REAL
   - plus_minus_home: INTEGER
   - video_available_home: INTEGER
   - team_id_away: TEXT
   - team_abbreviation_away: TEXT
   - team_name_away: TEXT
   - matchup_away: TEXT
   - wl_away: TEXT
   - fgm_away: REAL
   - fga_away: REAL
   - fg_pct_away: REAL
   - fg3m_away: REAL
   - fg3a_away: REAL
   - fg3_pct_away: REAL
   - ftm_away: REAL
   - fta_away: REAL
   - ft_pct_away: REAL
   - oreb_away: REA

#### Engineering the prompts based on above recommendations

In [ ]:
system_level_prompt=""" 
<instruction>
You are an expert data analyst that converts natural language questions into SQL on a National Basketball Association Database
</instruction> 

<instruction>
You only have access to a helper function defined below and no other external resources.

<helper_function>
def describe_table(table_name):
    return f"Returns the schema of the specified table from the NBA database, showing column names and types."
</helper_function>

</instruction>
"""

In [425]:
question_prompt="""
Convert the following question into a SQL query:

<question>{question}</question>
"""

In [426]:
query_type="""
This Natural Language question if of the following type:

<query_type>{query_type}</query_type>
"""

In [ ]:
few_shot_db_schema="""
For the purpose of this task, refer to the following schema of tables, the column names are included in the description:

<tables_list>
    <table>
        <table_name>team_info_common</table_name>
        <description>This table has details about individual teams like team_id, team_city, team_name</description>
    </table>
        <table_name>team_history</table_name> 
        <description>Refer to this table for details and has column names like year_founded, year_active_till</description>
    </table>
        <table_name>common_player_info</table_name>
        <description>This table has commonly asked details about individual players like person_id, team_name</description>
    </table>
        <table_name>game_info</table_name>
        <description>This table has commonly asked details about games like game_id, game_date</description>
    </table>
        <table_name>inactive_players</table_name>
        <description>This table has commonly asked details about inactive players like player_id</description>
    </table>
</tables_list>

"""

In [ ]:
precognition="""
Before you start writing the SQL, think about the question. Put your thinking into "" tags and return it in your response. 

When you are thinking, consider these points in the tags:

<instructions>
Refer to the schema to find which tables you need to query. If you do not find a table name, think about what the table name could be.
</instructions>

<instructions>
Once you have the table name, use the helper function to get column names.
</instructions>

<instructions>
Based on the query type provided above, think about the kind of output in the SELECT statement - single columns or multiple columns.
</instructions>
"""

In [ ]:
few_shot_examples="""
Here are some natural language to query examples you can refer to:

<example>
<question>What are the top 5 teams with the most championships?</question>
Thinking:
- I need to check the schema for a table that contains a column about championships.
- I use describe_table("team").
- Let's say it returns: id (int), full_name (test), championships_won (int)
- Now I can sort by championships_won and limit to 5.

<sql>
SELECT full_name FROM team ORDER BY championships_won DESC LIMIT 5;
</sql>
</example>

<example>
<question> List all players who are currently active and taller than 7 feet.</question>
<thinking>
- Use describe_table("common_player_info") to inspect columns.
- Assume it includes: id, name, height_in_inches, is_active
- Filter players where height > 84 inches and is_active = 1.
</thinking>

<sql>:
SELECT name FROM common_player_info WHERE height_in_inches > 84 AND is_active = 1;
</sql>
</example>
"""

#### We combine these prompts in a sequence that gives Claude enough sequential context. Here is one possible sequence:
- system_level_prompt: so that Claude knows its roles and constraints
- few_shot_db_schema: gives Claude a high level map of available tables/columns so that subsequent examples make sense
- few_shot_examples: Shows correct thinking patterns for Claude.
- precognition: gives step by step reasoning instructions before generating a SQL query
- query_type: This gives more context about the type of query
- main_prompt: This is the NL->SQL question.


- We will build the main prompt by concatenating the above sub-prompts. The main prompt is created in a way such that we can remove subprompts while prompt engineering without affecting the rest of the prompt

In [ ]:
main_prompt=""
if system_level_prompt:
    main_prompt+=f"""{system_level_prompt}"""
if few_shot_db_schema:
    main_prompt+=f"""{few_shot_db_schema}"""
if few_shot_examples:
    main_prompt+=f"""{few_shot_examples}"""
if precognition:
    main_prompt+=f"""{precognition}"""
if query_type:
    main_prompt+=f"""{precognition}"""
main_prompt+=question_prompt

#### We can call our function prompt_engineering_result which runs everything and gives us an evaluation of the final results

#### Trial 1

In [482]:
results,claude_sql_raw=prompt_engineering_result(main_prompt)


{
  "question": "What's the average weight of power forwards?",
  "ground_truth_sql": "SELECT ROUND(AVG(CAST(weight AS FLOAT)), 2) as avg_weight FROM common_player_info WHERE position LIKE '%F%' AND weight != ''",
  "gorund_truth_query_type": "aggregation",
  "claude_sql": "SELECT AVG(weight) AS average_weight\nFROM common_player_info\nWHERE position = 'PF';",
  "sql_semantic_match": false,
  "ground_truth": [
    220.04
  ],
  "claude_sql_result": [
    null
  ]
}
Overall accuracy: 0.0 %
{
  "question": "Which team has the newest arena?",
  "ground_truth_sql": "SELECT t.full_name FROM team t JOIN team_details td ON t.id = td.team_id ORDER BY td.arena DESC LIMIT 1",
  "gorund_truth_query_type": "detail",
  "claude_sql": "SELECT t.team_name, t.team_city, a.arena_name, a.year_built\nFROM team_info_common t\nJOIN team_arenas a ON t.team_id = a.team_id\nORDER BY a.year_built DESC\nLIMIT 1;",
  "sql_semantic_match": false,
  "ground_truth": [
    "Philadelphia 76ers"
  ],
  "claude_sql_resu

#### Checkout the claude_sql_raw to see how Claude was thinking
- While the accuracy still remains poor, we can also see that Claude was able to query correct table names in some cases.
- We see that letting Claude think gives it better direction to build a query

In [486]:
for i in claude_sql_raw[0:4]:
    print(i,"\n","#"*60,"\n")

I'll convert this natural language question into a SQL query.

"To answer this question about the average weight of power forwards, I need to:

1. Find a table that contains player positions and weights.
2. From the tables list, 'common_player_info' seems most relevant as it has 'commonly asked details about individual players'.
3. I would need to use the describe_table helper function to see if this table has columns for position and weight.
4. Since I don't have the actual ability to call the helper function, I'll assume common_player_info has columns like:
   - person_id
   - player_name
   - position
   - weight
5. I need to filter for players whose position is 'PF' (Power Forward) and calculate the average weight.
6. The output will be a single value - the average weight of power forwards."

```sql
SELECT AVG(weight) AS average_weight
FROM common_player_info
WHERE position = 'PF';
``` 
 ############################################################ 

I'll convert this natural langua

#### Testing out our new prompt on different types of queries
- Now we try running the above engineered prompt on just counting query types

In [490]:
counting_query_type_data = [i for i in ground_truth_data if i.get("type") == "counting"]

In [496]:
results_trial_2,claude_sql_raw_trial_2=prompt_engineering_result(main_prompt,counting_query_type_data)


{
  "question": "How many teams are in the NBA?",
  "ground_truth_sql": "SELECT COUNT(*) as team_count FROM team LIMIT 1",
  "gorund_truth_query_type": "counting",
  "claude_sql": "SELECT COUNT(*) AS total_teams\nFROM team_info_common;",
  "sql_semantic_match": false,
  "ground_truth": [
    30
  ],
  "claude_sql_result": [
    0
  ]
}
Overall accuracy: 15.79 %
{
  "question": "What's the total number of games played?",
  "ground_truth_sql": "SELECT COUNT(DISTINCT game_id) as total_games FROM game LIMIT 1",
  "gorund_truth_query_type": "counting",
  "claude_sql": "SELECT COUNT(*) AS total_games\nFROM game_info;",
  "sql_semantic_match": false,
  "ground_truth": [
    65642
  ],
  "claude_sql_result": [
    58053
  ]
}
Overall accuracy: 15.79 %


In [499]:
round(results_trial_2['accuracy']*100,2)

15.79

- We see a ~16% lift in accuracy for Counting based queries in Trial 2 using our prompt engineering

**Next steps**
- We try out aggregation type queries now
- Based on this lift, it is possible that more advanced queries generated by Claude like filtering, ranking (especially because they could require joins) are not performing as well on the DB.

In [516]:
aggregation_query_type_data = [i for i in ground_truth_data if i.get("type") == "aggregation"]
results_trial_3,claude_sql_raw_trial_3=prompt_engineering_result(main_prompt,aggregation_query_type_data)


{
  "question": "What's the average points per game?",
  "ground_truth_sql": "SELECT ROUND(AVG(pts_home + pts_away) / 2, 2) as avg_points FROM game LIMIT 1",
  "gorund_truth_query_type": "aggregation",
  "claude_sql": "SELECT AVG(points) AS average_points_per_game\nFROM game_stats;",
  "sql_semantic_match": false,
  "ground_truth": [
    102.81
  ],
  "claude_sql_result": null
}
Overall accuracy: 0.0 %
{
  "question": "What's the average height of NBA players?",
  "ground_truth_sql": "SELECT ROUND(AVG(CAST(SUBSTR(height, 1, INSTR(height, '-')-1) AS FLOAT)), 2) as avg_height FROM common_player_info WHERE height != '' LIMIT 1",
  "gorund_truth_query_type": "aggregation",
  "claude_sql": "SELECT AVG(height) AS average_height\nFROM common_player_info;",
  "sql_semantic_match": false,
  "ground_truth": [
    6.03
  ],
  "claude_sql_result": [
    5.910242290748899
  ]
}
Overall accuracy: 0.0 %


- In the above results for avg weight, we see that Claude was close to actual queries, and we can add additional metrics like rounding in aggregration based queries in our precognition.

### Trial 2
- We also add thinking for primary-keys that could be foreign-key pairs for queries requiring joins work, we give a few shot example for join. We keep just two examples to not make the full prompt too verbose. However, keeping the earlier 2 and adding a new one could also change our results. Feel free to try it out!
- We also add rounding elements for aggregation type queries

In [ ]:
precognition="""
Before you start writing the SQL, think about the question. Put your thinking into "" tags and return it in your response. 

When you are thinking, consider these points in the tags:

<instructions>
Refer to the schema to find which tables you need to query. If you do not find a table name, think about what the table name could be.
</instructions>

<instructions>
Once you have the table name, use the helper function to get column names.
</instructions>

<instructions>
Think about tables that might need to be joined 
</instructions>

<instructions>
Based on the query type provided above, think about the kind of output in the SELECT statement - single columns or multiple columns.
</instructions>

<instructions>
If the query type is <query_type>aggregation</query_type> then round your outputs to two decimal places.
</instructions>
"""

In [521]:
few_shot_examples="""
Here are some natural language to query examples you can refer to:

<example>
<question>What are the top 5 teams with the most championships?</question>
Thinking:
- I need to check the schema for a table that contains a column about championships.
- I use describe_table("team").
- Let's say it returns: id (int), full_name (test), championships_won (int)
- Now I can sort by championships_won and limit to 5.

<sql>
SELECT full_name FROM team ORDER BY championships_won DESC LIMIT 5;
</sql>
</example>

<example>
<question>List all players and their teams for the 2022 season.</question>
<thinking>
- Use describe_table("common_player_info") and describe_table("team") to inspect structure.
- Assume common_player_info includes: player_id, season, team_id
- Assume team includes: id, full_name
- Join on team_id = team.id and filter by season = 2022.
</thinking>
<sql>
SELECT p.player_id, t.full_name 
FROM player_season_stats p
JOIN team t ON p.team_id = t.id
WHERE p.season = 2022;
</sql>
</example>
"""

In [523]:
main_prompt=""
if system_level_prompt:
    main_prompt+=f"""{system_level_prompt}"""
if few_shot_db_schema:
    main_prompt+=f"""{few_shot_db_schema}"""
if few_shot_examples:
    main_prompt+=f"""{few_shot_examples}"""
if precognition:
    main_prompt+=f"""{precognition}"""
if query_type:
    main_prompt+=f"""{precognition}"""
main_prompt+=question_prompt


We run the same on aggregation query type to see if there is any lift

In [ ]:
results_trial_4,claude_sql_raw_trial_4=prompt_engineering_result(main_prompt,aggregation_query_type_data)


#### Next steps:

- Your analytics team can try out this setup and engineer prompts to achieve accuracy lifts.